# Rec Sys of Answerers for StackOverflow
## Version 1.0

Loading Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import norm
from sklearn.model_selection import train_test_split
import ast


Loading Data

In [2]:
df_question = pd.read_csv("dataset/questions_2019.csv")
df_answer = pd.read_csv("dataset/answers_2019.csv")

In [3]:
df_question.head()

,QuestionId,QuestionOwnerId,QuestionTitle,QuestionTags,QuestionVotes,QuestionCreationDate,AnswerCount
0,54936100,3419772,R 3.5.2: Error in loading stock data from zoo,['r'],0,2019-03-01 00:00:10,1
1,54936106,3997132,Different behaviour of range with lodash/fp,"['functional-programming', 'lodash']",1,2019-03-01 00:01:48,1
2,54936108,4992551,webPack dev server proxy rewrite URLs in response,"['webpack', 'webpack-dev-server']",2,2019-03-01 00:01:55,1
3,54936109,2239552,EF Core 2.0 Global Filter,['c#'],0,2019-03-01 00:02:36,0
4,54936112,5505171,Clustered column chart in C# using Chart in Wi...,"['c#', 'winforms', 'charts', 'column-chart']",0,2019-03-01 00:02:47,0


In [4]:
df_answer.head()

,QuestionId,AnswerOwnerId,AnswerVotes,AnswerCreationDate
0,56140111,10245958,3,2019-05-15 00:07:23
1,56140157,1226963,0,2019-05-15 00:12:34
2,56140125,6841773,0,2019-05-15 00:12:44
3,56140150,1440565,5,2019-05-15 00:12:52
4,56140150,11015427,15,2019-05-15 00:16:46


Transfoming list of tags

In [5]:
df_question['QuestionTags'] = df_question['QuestionTags'].apply(ast.literal_eval)

In [6]:
df_question

,QuestionId,QuestionOwnerId,QuestionTitle,QuestionTags,QuestionVotes,QuestionCreationDate,AnswerCount
0,54936100,3419772,R 3.5.2: Error in loading stock data from zoo,[r],0,2019-03-01 00:00:10,1
1,54936106,3997132,Different behaviour of range with lodash/fp,"[functional-programming, lodash]",1,2019-03-01 00:01:48,1
2,54936108,4992551,webPack dev server proxy rewrite URLs in response,"[webpack, webpack-dev-server]",2,2019-03-01 00:01:55,1
3,54936109,2239552,EF Core 2.0 Global Filter,[c#],0,2019-03-01 00:02:36,0
4,54936112,5505171,Clustered column chart in C# using Chart in Wi...,"[c#, winforms, charts, column-chart]",0,2019-03-01 00:02:47,0
...,...,...,...,...,...,...,...
878713,54272409,10245420,Selenium iframe Data issue - python,"[python, selenium, selenium-webdriver, iframe]",1,2019-01-19 23:57:14,1
878714,54272411,8906835,Problem on adding Bootstrap carousel component,"[bootstrap-4, bootstrap-carousel]",0,2019-01-19 23:57:26,1
878715,54272412,10893334,NativeScript Vue nativescript-sqlite cannot as...,"[sqlite, nativescript, nativescript-vue]",0,2019-01-19 23:57:36,1
878716,54272414,10564619,How to fix 'TypeError' that comes up for some ...,"[python, typeerror]",1,2019-01-19 23:57:54,1


## Prototype 1 - Approach Collaborative Filtering


- **Goal:** Find answerers to new posted question.
  
- **Approach:** Collaborative Filtering (Item Based) --> Utilize question tags of previous answered questions, to indicate the most appropriate user.

- **Steps:**

1. Create Matrix of User-Tags
2. Calculate cosine similarity between tags
3. Calculate the the score of the user given the number of times they answerered certain question and the similarity of that tag with the the tags of the new question.

### Dividing the question dataset into Train and Test

In [7]:
train_data, test_data = train_test_split(df_question, test_size=0.2, random_state=42)

Merging QuestionTags to UserAnswerId

In [8]:
df_merged = train_data[["QuestionId","QuestionTags"]].merge(df_answer[["QuestionId", "AnswerOwnerId"]], how='inner', on = "QuestionId", )

Getting all tags of a user -> for checking purposes

In [9]:
df_grouped_tags = (
    df_merged.groupby("AnswerOwnerId")["QuestionTags"]
    .apply(lambda tags: list(set(tag for sublist in tags for tag in sublist)))  # Flatten and deduplicate tags
    .reset_index()
)

df_grouped_tags.columns = ["AnswerOwnerId", "CombinedTags"]

In [10]:
df_grouped_tags[df_grouped_tags["AnswerOwnerId"] == 1185]["CombinedTags"].iloc[0]

['r']

In [11]:
1943329

1943329

### Indexing tags and users

In [12]:
dic_merged = df_merged.to_dict(orient="records")

In [13]:
unique_tags = list({tag for question in dic_merged for tag in question['QuestionTags']})
unique_users = list({question['AnswerOwnerId'] for question in dic_merged})

tag_to_index = {tag: idx for idx, tag in enumerate(unique_tags)}
user_to_index = {user: idx for idx, user in enumerate(unique_users)}

### Creating User-Tag Matrix

In [14]:
# Step 2: Create a Sparse User-Tag Matrix
rows, cols, data = [], [], []
for question in dic_merged:
    user_idx = user_to_index[question['AnswerOwnerId']]
    for tag in question['QuestionTags']:
        tag_idx = tag_to_index[tag]
        rows.append(user_idx)
        cols.append(tag_idx)
        data.append(1)

# Create a sparse matrix 
user_tag_matrix_sparse = csr_matrix((data, (rows, cols)), shape=(len(unique_users), len(unique_tags)))


### Calculating Tags Similarity Matrix

In [15]:
# Compute Tag Similarity
cos_sim_matrix = cosine_similarity(user_tag_matrix_sparse.T)

### Recommending User

In [16]:
test_data

,QuestionId,QuestionOwnerId,QuestionTitle,QuestionTags,QuestionVotes,QuestionCreationDate,AnswerCount
275130,56705914,11659955,How to style my react select when it is disabl...,"[javascript, html, css, reactjs]",0,2019-06-21 15:00:47,1
623441,55720363,2579263,Serializing into dictionary Stored Procedure r...,"[python, stored-procedures, flask, sqlalchemy,...",0,2019-04-17 05:17:42,1
603387,55655622,11199969,Refresh page from another (page),"[javascript, php, jquery, html, ajax]",3,2019-04-12 16:05:10,2
351087,56204717,11519243,Slow loading web page served by Node.js HTTP s...,"[node.js, video.js, httpserver]",1,2019-05-19 04:18:20,1
44361,55080751,11176740,Castle Windsor 5.0 PerWebRequest Lifestyle,[asp.net],2,2019-03-09 18:39:45,1
...,...,...,...,...,...,...,...
511137,55527860,6112457,"Tee to commands only, not stdout","[bash, tee]",2,2019-04-05 03:36:57,2
577601,56576970,10801227,How to sort data in Scala?,[scala],0,2019-06-13 09:01:37,2
634629,55756542,1258629,How to serve static resources in Yada,"[clojure, yada]",1,2019-04-19 04:16:49,2
235126,55829103,9102628,How to group criteria or elements in MS Access,"[database, ms-access]",0,2019-04-24 11:33:07,1


In [17]:
## Example to 1 question
# tags = test_data["QuestionTags"].iloc[0]
# tags

tags = ["r", "radar-chart", "spider-chart"]

In [18]:
def recommend_users_collaborative(tags, top_n=10):
    # Get indices of the tags
    tag_indices = [tag_to_index[tag] for tag in tags if tag in tag_to_index]
    if len(tag_indices)==0:
        return [] ## No tag found to make the suggestion
    
    # Combine similarities for the requested tags
    similarity_tags = [cos_sim_matrix[:, tag_index] for tag_index in tag_indices]
    combined_similarity = sum(similarity_tags)

    # Compute user scores 
    user_scores_vector = user_tag_matrix_sparse.dot(combined_similarity)

    # Map scores to users
    user_scores = {unique_users[idx]: score for idx, score in enumerate(user_scores_vector)}

    # Rank users by scores
    ranked_users = sorted(user_scores.items(), key=lambda x: x[1], reverse=True)
    # return ranked_users[:top_n]
    return ranked_users

In [19]:
return_1 = recommend_users_collaborative(tags)

## Prototype 2 - Approach Content Based-Filtering


- **Goal:** Find answerers to new posted question.
  
- **Approach:** Content Based-Filtering --> Utilize question tags of previous answered questions, to indicate the most appropriate user.

- **Steps:**

1. Create Matrix of User-Tags
2. Create vector of to the new tag based on the Matrix of User-tags 
3. Calculate cosine similarity for each user with vector created in step 2
4. Order list to show the most recommended users

In [20]:
# We are going start using some User-Tag matrix of the previous approach

def recommend_users_content(tags, top_n=10):
    # Get index of the tags
    tag_indices = [tag_to_index[tag] for tag in tags if tag in tag_to_index]
    if len(tag_indices)==0:
        return [] ## No tag found to make the suggestion
            
    # Create vector of the tags in question
    question_vector = np.zeros(user_tag_matrix_sparse.shape[1])
    question_vector[tag_indices] = 1
    question_vector= question_vector.reshape(1, -1) #Reshaping array to calculate similarity

    user_scores_vector = cosine_similarity(user_tag_matrix_sparse, question_vector)
    user_scores_vector = user_scores_vector.flatten()

    # Map scores to users
    user_scores = {unique_users[idx]: score for idx, score in enumerate(user_scores_vector)}

    # Rank users by scores
    ranked_users = sorted(user_scores.items(), key=lambda x: x[1], reverse=True)
    # return ranked_users[:10]
    return ranked_users


In [21]:
return_2 = recommend_users_content(tags)

## Evaluation

Top-N Accuracy

In [22]:
df_merged_test = test_data[["QuestionId","QuestionTags"]].merge(df_answer[["QuestionId", "AnswerOwnerId"]], how='inner', on = "QuestionId", )

Joining the Users to form a single ground truth

In [23]:
# Convert QuestionTags from lists to tuples
df_merged_test["QuestionTags"] = df_merged_test["QuestionTags"].apply(tuple)

# Group AnswerOwnerId by QuestionId and QuestionTags
grouped_df = (
    df_merged_test.groupby(["QuestionId", "QuestionTags"])["AnswerOwnerId"]
    .apply(list)  # Combine AnswerOwnerId values into a list
    .reset_index()
    .rename(columns={"AnswerOwnerId": "GroundTruth"})
)

In [24]:
ground_truth_dic = grouped_df.to_dict(orient='records')

Calculate the results for all questions

In [25]:
def top_n_accuracy(ground_truth_dic, reccomendation_function, top_n = 5):
    tp = 0
    for question in ground_truth_dic:    
        #Getting recommendation
        top_users = reccomendation_function(question["QuestionTags"])
        top_users_list = [user[0] for user in top_users[:top_n]]
        
        # Checking if it is in the answer
        users_groundtruth = question["GroundTruth"]
        if set(top_users_list) & set(users_groundtruth):
            tp += 1
            
    return tp/len(ground_truth_dic)


Collaborative

In [39]:
top_n = [5, 10, 20, 50]
results = []

for top in top_n:
    results.append({
        "acc": top_n_accuracy(ground_truth_dic[:100], recommend_users_collaborative, top),
        "top": top,
        "type": "collaborative"
    })

Content

In [40]:
results

[{'acc': 0.1, 'top': 5, 'type': 'collaborative'},
 {'acc': 0.13, 'top': 10, 'type': 'collaborative'},
 {'acc': 0.19, 'top': 20, 'type': 'collaborative'},
 {'acc': 0.28, 'top': 50, 'type': 'collaborative'}]